In [2]:
import numpy as np
import cv2 as cv

In [3]:
def sobel_func(image):
    dx_img = cv.Sobel(src=image, ddepth=cv.CV_16S, dx=1, dy=0, ksize=3)
    dy_img = cv.Sobel(src=image, ddepth=cv.CV_16S, dx=0, dy=1, ksize=3)

    Xaxis_img = cv.convertScaleAbs(src=dx_img)
    Yaxis_img = cv.convertScaleAbs(src=dy_img)
    sobel_img = cv.addWeighted(src1=Xaxis_img, alpha=0.5, src2=Yaxis_img,beta=0.5, gamma=0)
    return sobel_img

In [4]:
# 開啟攝影機
# 函式輸出入 => 攝影機 = cv.VideoCapture(攝影機編號)
cap = cv.VideoCapture(0)

# 設定擷取影像的尺寸大小
cap.set(cv.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 360)

# 使用while迴圈不斷從攝影機獲取新影像
while(True):
    # 讀取攝影機影像
    # 函式輸出入 => 是否正常讀取, 攝影機影像 = cv.VideoCapture.read()
    ret, frame = cap.read()
    
    # 若正常讀取攝影機影像則將影像寫入影片
    # 函式輸出入 => cv.VideoWriter.write(欲寫入的影像)
    if ret == False:
        print("Camera read failed!")
        break
        
    # 影像灰階
    gframe = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #sobel
    sobel_frame = sobel_func(gframe)
    sobel_frame = cv.cvtColor(sobel_frame, cv.COLOR_GRAY2BGR)
    
    #laplacian
    lp_frame = cv.Laplacian(src=gframe, ddepth=-1, ksize=3)
    lp_frame = cv.cvtColor(lp_frame, cv.COLOR_GRAY2BGR)
    
#     canny
    canny_frame = cv.Canny(gframe, 50, 150)
    canny_frame = cv.cvtColor(canny_frame, cv.COLOR_GRAY2BGR)
        
    # 顯示攝影機影像
    # 函式輸出入 => cv.imshow(視窗名稱, 影像矩陣)
    result = cv.hconcat([frame, sobel_frame, lp_frame, canny_frame]) 
    
    cv.imshow('edge', result)

    # 若按下ESC鍵則離開迴圈
    # 函式輸出入 => 按鍵字元編碼 = cv.waitKey(停等時間單位毫秒)
    if cv.waitKey(1) & 0xFF == 27:
        break

# 關閉所有OpenCV的視窗
# 函式輸出入 => cv.destroyAllWindows()
cap.release()
cv.destroyAllWindows()


In [5]:
#定義LBP演算法的函數
def lbp(img):
    assert(len(img.shape) == 2) # LBP只接受灰階影像
    ret = np.zeros_like(img)
    
    # 將圖片擴大，為了可以處理邊界
    img = cv.copyMakeBorder(img, 1, 1, 1, 1, cv.BORDER_REPLICATE)
    
    for y in range(1, img.shape[0] - 1):
        for x in range(1, img.shape[1] - 1):
            center = img[y][x]
            pixel = 0
            pixel |= (img[y - 1][x - 1] >= center) << 0
            pixel |= (img[y - 1][x + 0] >= center) << 1
            pixel |= (img[y - 1][x + 1] >= center) << 2
            pixel |= (img[y + 0][x + 1] >= center) << 3
            pixel |= (img[y + 1][x + 1] >= center) << 4
            pixel |= (img[y + 1][x + 0] >= center) << 5
            pixel |= (img[y + 1][x - 1] >= center) << 6
            pixel |= (img[y + 0][x - 1] >= center) << 7
            
            ret[y-1][x-1] = pixel
    return ret

In [6]:
# 開啟攝影機
# 函式輸出入 => 攝影機 = cv.VideoCapture(攝影機編號)
cap = cv.VideoCapture(0)

# 設定擷取影像的尺寸大小
cap.set(cv.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 360)

# 使用while迴圈不斷從攝影機獲取新影像
while(True):
    # 讀取攝影機影像
    # 函式輸出入 => 是否正常讀取, 攝影機影像 = cv.VideoCapture.read()
    ret, frame = cap.read()
    
    # 若正常讀取攝影機影像則將影像寫入影片
    # 函式輸出入 => cv.VideoWriter.write(欲寫入的影像)
    if ret == False:
        print("Camera read failed!")
        break
        
    # 影像灰階
    gframe = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

#     lbp
    lbp_frame = lbp(gframe)
    lbp_frame = cv.cvtColor(lbp_frame, cv.COLOR_GRAY2BGR)
    
    # 顯示攝影機影像
    # 函式輸出入 => cv.imshow(視窗名稱, 影像矩陣)
    result = cv.hconcat([frame, lbp_frame]) 
    
    cv.imshow('edge', result)

    # 若按下ESC鍵則離開迴圈
    # 函式輸出入 => 按鍵字元編碼 = cv.waitKey(停等時間單位毫秒)
    if cv.waitKey(1) & 0xFF == 27:
        break

# 關閉所有OpenCV的視窗
# 函式輸出入 => cv.destroyAllWindows()
cap.release()
cv.destroyAllWindows()